## Configuration and Setup


In [ ]:
import os
os.environ['HF_HOME'] = "C:/HF_CACHE/"

In [ ]:
import torch

from cs324_project.datasets import GlueDatasetTask, load_glue_dataset_info
from cs324_project.models import ModelCheckpointName, load_classification_model, load_pretraining_model, load_tokenizer
from cs324_project.masking import (
    get_training_args_mlm, get_trainer_mlm, RandomMaskingConfig, WholeWordMaskingConfig, TyphoonMaskingConfig)
from cs324_project.classification import get_training_args_sc, get_trainer_sc

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model_name = ModelCheckpointName.TINYBERT_HUAWEI
task = GlueDatasetTask.MRPC

In [ ]:
tokenizer = load_tokenizer(model_name)
dataset_info = load_glue_dataset_info(task, tokenizer)

## Fine-tune with masking

In [ ]:
model_mlm = load_pretraining_model(model_name, dataset_info)

In [ ]:
training_args_mlm = get_training_args_mlm(
    masking_config=RandomMaskingConfig(),
    num_epochs=100)
trainer_mlm = get_trainer_mlm(
    dataset_info=dataset_info,
    mlm_args=training_args_mlm,
    model=model_mlm)

In [ ]:
trainer_mlm.train()

In [ ]:
trainer_mlm.evaluate()

In [ ]:
print(trainer_mlm.state.best_model_checkpoint)

## Fine-tune on sequence classification task

In [ ]:
model_sc = load_classification_model(trainer_mlm.state.best_model_checkpoint, dataset_info)

In [ ]:
training_args_sc = get_training_args_sc(
    task,
    num_epochs=100)
trainer_sc = get_trainer_sc(
    dataset_info=dataset_info,
    model=model_sc,
    training_args=training_args_sc)

In [ ]:
trainer_sc.train()

In [ ]:
trainer_sc.evaluate()

In [ ]:
print(trainer_sc.state.best_model_checkpoint)